In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4275954498072227738
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14378926080
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12633255073414031756
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/My Drive/Covid/Covid19-dataset'

/content/drive/My Drive/Covid/Covid19-dataset


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from sklearn import preprocessing
import glob
import cv2
import os

print (os.listdir("/content/drive/My Drive/Covid/Covid19-dataset/"))
train_images = []
train_labels = []
for directory_path in glob.glob("/content/drive/My Drive/Covid/Covid19-dataset/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (128,128))
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_images = np.ndarray(shape=(train_images.shape[0], train_images.shape[1], train_images.shape[2], 3), dtype= np.uint8) 
train_images[:, :, :, 0] = train_images[:, :, :, 0]
train_images[:, :, :, 1] = train_images[:, :, :, 0]
train_images[:, :, :, 2] = train_images[:, :, :, 0]
train_labels = np.array(train_labels)

print(train_images.shape)
train_labels = np.array(train_labels)

test_images = []
test_labels = []
for directory_path in glob.glob("/content/drive/My Drive/Covid/Covid19-dataset/test/*"):
    nodule_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (128,128))
        test_images.append(img)
        test_labels.append(nodule_label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

#encode labels from text to integers.
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#split data into test and train datasets
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded
#Normalize
x_train, x_test = x_train / 255.0, x_test / 255.0
print('Train images shape:', x_train.shape)
#one hot encoded y values for neural network
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

['train', 'test']
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(115).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(112).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(102).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(110).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(107).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(1).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(108).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(10).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(0).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(111).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(109).jpg
/content/drive/My Drive/Covid/Covid19-dataset/train/COVID19/COVID19(11).jpg
/con

In [ ]:
from keras.layers import Input 
from keras.applications.inception_v3 import InceptionV3
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model, load_model
from keras.layers import Input, InputLayer, Conv2D, Dense, MaxPooling2D, SeparableConv2D, BatchNormalization, ZeroPadding2D, GlobalAveragePooling2D,Flatten,Average, Dropout

input_tensor = Input(shape=(128,128,1) )
model_input = Conv2D(3,(3,3),padding='same')(input_tensor)
def inceptionv3_cnn(model_input):
    inceptionv3_cnn = InceptionV3(weights='imagenet', include_top=False)
    x = inceptionv3_cnn.output
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(1024, (3, 3), activation='relu', name='extra_conv_inceptionv3')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=inceptionv3_cnn.input, outputs=predictions, name='inceptionv3_custom')
    return model

#instantiate the model
inceptionv3_custom_model = inceptionv3_cnn(model_input)

#display model summary
inceptionv3_custom_model.summary() 

87910968/87910968 [==============================] - 5s 0us/step
Model: "inceptionv3_custom"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d_1[0][0]']               
 alization)     

In [ ]:
inceptionv3_custom_model.compile(
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
loss = tf.keras.losses.CategoricalCrossentropy(),
metrics = ['accuracy'])
batch_size = 16
history = inceptionv3_custom_model.fit(x_train, y_train_one_hot, epochs=100)

Epoch 1/100
55/55 [==============================] - 37s 116ms/step - loss: 0.5791 - accuracy: 0.7362
Epoch 2/100
55/55 [==============================] - 6s 112ms/step - loss: 0.5853 - accuracy: 0.7362
Epoch 3/100
55/55 [==============================] - 6s 114ms/step - loss: 0.5818 - accuracy: 0.7362
Epoch 4/100
55/55 [==============================] - 6s 113ms/step - loss: 0.5845 - accuracy: 0.7362
Epoch 5/100
55/55 [==============================] - 6s 115ms/step - loss: 0.5805 - accuracy: 0.7362
Epoch 6/100
55/55 [==============================] - 6s 114ms/step - loss: 0.5776 - accuracy: 0.7362
Epoch 7/100
55/55 [==============================] - 6s 115ms/step - loss: 0.5822 - accuracy: 0.7362
Epoch 8/100
55/55 [==============================] - 6s 114ms/step - loss: 0.5839 - accuracy: 0.7362
Epoch 9/100
55/55 [==============================] - 6s 115ms/step - loss: 0.5777 - accuracy: 0.7362
Epoch 10/100
55/55 [==============================] - 6s 115ms/step - loss: 0.5838 - accur